# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [508]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Generate the mathematical model

In [544]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [545]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Sets and Indices:**

*  *I* = Set of power plants, *I* = {1, 2, 3, 4, 5, 6}
* *T* = Set of time periods, *T* = {1, 2, ..., 15}

**Parameters:**

*  *SU<sub>i</sub>* = Startup cost for power plant *i* (€), *i* ∈ *I*
    *  *SU<sub>1</sub>* = 10324, *SU<sub>2</sub>* = 5678, *SU<sub>3</sub>* = 7802, *SU<sub>4</sub>* = 12899, *SU<sub>5</sub>* = 4596, *SU<sub>6</sub>* = 9076
* *SD<sub>i</sub>* = Shutdown cost for power plant *i* (€), *i* ∈ *I*
    * *SD<sub>1</sub>* = 2673, *SD<sub>2</sub>* = 5893, *SD<sub>3</sub>* = 982, *SD<sub>4</sub>* = 6783, *SD<sub>5</sub>* = 2596, *SD<sub>6</sub>* = 3561
* *FC<sub>i</sub>* = Fixed cost for running power plant *i* (€/period), *i* ∈ *I* 
* *VC<sub>it</sub>* = Variable cost for power plant *i* in time period *t* (€/MWh), *i* ∈ *I*, *t* ∈ *T*
* *L<sub>i</sub>* = Lower bound for output power of power plant *i* (MW), *i* ∈ *I*
* *U<sub>i</sub>* = Upper bound for output power of power plant *i* (MW), *i* ∈ *I*
* *ΔP<sub>i</sub><sup>+</sup>* = Maximum power increment for power plant *i* (MW), *i* ∈ *I*
* *ΔP<sub>i</sub><sup>-</sup>* = Maximum power decrement for power plant *i* (MW), *i* ∈ *I*
* *D<sub>t</sub>* = Total power demand in time period *t* (MW), *t* ∈ *T*

**Decision Variables:**

* *P<sub>it</sub>* = Power output of power plant *i* in time period *t* (MW), *i* ∈ *I*, *t* ∈ *T*
* *x<sub>it</sub>* = Binary variable indicating if power plant *i* is running in time period *t*, *i* ∈ *I*, *t* ∈ *T* (1 if running, 0 otherwise)
* *SU<sub>it</sub>* = Binary variable indicating if power plant *i* is started up in time period *t*, *i* ∈ *I*, *t* ∈ *T* (1 if started, 0 otherwise)
* *SD<sub>it</sub>* = Binary variable indicating if power plant *i* is shut down in time period *t*, *i* ∈ *I*, *t* ∈ *T* (1 if shut down, 0 otherwise)

**Objective Function:**

Minimize the total cost:

```
Minimize ∑<sub>i∈I</sub> ∑<sub>t∈T</sub> (FC<sub>i</sub>*x<sub>it</sub> + VC<sub>it</sub>*P<sub>it</sub> + SU<sub>i</sub>*SU<sub>it</sub> + SD<sub>i</sub>*SD<sub>it</sub>)
```

**Constraints:**

1. **Power balance:** The total power output should meet the demand with a 10% security margin:
   ```
   ∑<sub>i∈I</sub> P<sub>it</sub> ≥ 1.1 * D<sub>t</sub>, ∀t ∈ T
   ```

2. **Output bounds:** The output of each power plant should be within its limits:
   ```
   L<sub>i</sub>*x<sub>it</sub> ≤ P<sub>it</sub> ≤ U<sub>i</sub>*x<sub>it</sub>, ∀i ∈ I, ∀t ∈ T
   ```

3. **Ramp rate limits:** The change in output power between periods is limited:
   ```
   -ΔP<sub>i</sub><sup>-</sup> ≤ P<sub>it</sub> - P<sub>i(t-1)</sub> ≤ ΔP<sub>i</sub><sup>+</sup>, ∀i ∈ I, ∀t ∈ T, t > 1
   ```

4. **Startup and shutdown logic:** 
   ```
   SU<sub>it</sub> ≥ x<sub>it</sub> - x<sub>i(t-1)</sub>, ∀i ∈ I, ∀t ∈ T, t > 1
   SD<sub>it</sub> ≥ x<sub>i(t-1)</sub> - x<sub>it</sub>, ∀i ∈ I, ∀t ∈ T, t > 1
   ```

5. **Binary variable definition:**
   ```
   x<sub>it</sub>, SU<sub>it</sub>, SD<sub>it</sub> ∈ {0, 1}, ∀i ∈ I, ∀t ∈ T 
   ``` 

This mathematical optimization model can be solved using suitable optimization software to find the optimal power output schedule for the six power plants over the 15 time periods, minimizing the total cost while satisfying all the operational constraints. 


## 3. Generate the pyomo code

In [546]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [547]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982,  4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 120, 5: 70, 6: 110}  # Sample fixed costs
VC = {(i, t): 0.1*i for i in range(1, 7) for t in range(1, 16)}  # Sample variable costs
L = {1: 10, 2: 8, 3: 12, 4: 15, 5: 9, 6: 11}  # Sample lower bounds
U = {1: 50, 2: 40, 3: 45, 4: 60, 5: 35, 6: 55}  # Sample upper bounds
Delta_P_plus = {1: 5, 2: 4, 3: 6, 4: 8, 5: 3, 6: 7}  # Sample max power increment
Delta_P_minus = {1: 5, 2: 4, 3: 6, 4: 8, 5: 3, 6: 7}  # Sample max power decrement
D = {t: 100 + 10*t for t in range(1, 16)}  # Sample demand data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Set of power plants
model.T = pyo.Set(initialize=range(1, 16))  # Set of time periods

# Parameters (already defined as sample data above)
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.L = pyo.Param(model.I, initialize=L)
model.U = pyo.Param(model.I, initialize=U)
model.Delta_P_plus = pyo.Param(model.I, initialize=Delta_P_plus)
model.Delta_P_minus = pyo.Param(model.I, initialize=Delta_P_minus)
model.D = pyo.Param(model.T, initialize=D)

# Decision variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.x = pyo.Var(model.I, model.T, within=pyo.Binary)  # Running status
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status

# Objective function
def objective_rule(model):
    return sum(model.FC[i] * model.x[i, t] + model.VC[i, t] * model.P[i, t] + model.SU[i] * model.SU_var[i, t] + model.SD[i] * model.SD_var[i, t] 
               for i in model.I for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

def output_bounds_rule(model, i, t):
    return model.L[i] * model.x[i, t] <= model.P[i, t] <= model.U[i] * model.x[i, t]
model.output_bounds = pyo.Constraint(model.I, model.T, rule=output_bounds_rule)

def ramp_rate_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] <= model.Delta_P_plus[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_up = pyo.Constraint(model.I, model.T, rule=ramp_rate_up_rule)

def ramp_rate_down_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] >= -model.Delta_P_minus[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_down = pyo.Constraint(model.I, model.T, rule=ramp_rate_down_rule)

def startup_rule(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.x[i, t] - model.x[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.x[i, t-1] - model.x[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Optimal Solution:")
for i in model.I:
    for t in model.T:
        print(f"Power Plant {i}, Time {t}: Output = {pyo.value(model.P[i, t])}, Status = {'Running' if pyo.value(model.x[i, t]) == 1 else 'Off'}")
```

**Explanation of the code and sample data:**

1.  **Import pyomo.environ as pyo:** Imports the Pyomo library.
2.  **Sample Data:**  This section defines the sample data for all the sets and parameters. This is where you would replace the sample values with your actual data.  
3.  **Model Creation:** A ConcreteModel is created to represent the optimization problem.
4.  **Sets:** Defines the sets of power plants (`model.I`) and time periods (`model.T`).
5.  **Parameters:** Defines all the parameters using the provided sample data.
6.  **Decision Variables:** Defines the variables of the model:
    *   `model.P`: Power output of each power plant at each time period.
    *   `model.x`:  Binary variable indicating whether a power plant is running.
    *   `model.SU_var`: Binary variable indicating startup.
    *   `model.SD_var`: Binary variable indicating shutdown.
7.  **Objective Function:** Defines the objective function to minimize total cost, as described in the mathematical model.
8.  **Constraints:** Defines all the constraints:
    *   `model.power_balance`: Ensures total power output meets demand plus security margin.
    *   `model.output_bounds`: Ensures power output is within the bounds of each plant.
    *   `model.ramp_rate_up` and `model.ramp_rate_down`: Enforces ramp rate limits.
    *   `model.startup` and `model.shutdown`: Defines startup and shutdown logic using binary variables.
9.  **Solve the Model:**
    *   `solver = pyo.SolverFactory('cbc')`: Selects the CBC solver (you can change this to other solvers).
    *   `results = solver.solve(model)`: Solves the model using the selected solver.
10. **Print Results:** Prints the solver results and the optimal power output and running status of each power plant at each time period. 

Remember that this code uses sample data, so you need to replace it with your specific data for a meaningful solution. Also, choose a solver that is appropriate for your problem and computing environment. 


## 4. Input problem data and try running the generated code

In [548]:
import pyomo.environ as pyo

# Sample data (replace with actual data)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982,  4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 120, 5: 70, 6: 110}  # Sample fixed costs
VC = {(i, t): 0.1*i for i in range(1, 7) for t in range(1, 16)}  # Sample variable costs
L = {1: 10, 2: 8, 3: 12, 4: 15, 5: 9, 6: 11}  # Sample lower bounds
U = {1: 50, 2: 40, 3: 45, 4: 60, 5: 35, 6: 55}  # Sample upper bounds
Delta_P_plus = {1: 5, 2: 4, 3: 6, 4: 8, 5: 3, 6: 7}  # Sample max power increment
Delta_P_minus = {1: 5, 2: 4, 3: 6, 4: 8, 5: 3, 6: 7}  # Sample max power decrement
D = {t: 100 + 10*t for t in range(1, 16)}  # Sample demand data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Set of power plants
model.T = pyo.Set(initialize=range(1, 16))  # Set of time periods

# Parameters (already defined as sample data above)
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.L = pyo.Param(model.I, initialize=L)
model.U = pyo.Param(model.I, initialize=U)
model.Delta_P_plus = pyo.Param(model.I, initialize=Delta_P_plus)
model.Delta_P_minus = pyo.Param(model.I, initialize=Delta_P_minus)
model.D = pyo.Param(model.T, initialize=D)

# Decision variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.x = pyo.Var(model.I, model.T, within=pyo.Binary)  # Running status
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status

# Objective function
def objective_rule(model):
    return sum(model.FC[i] * model.x[i, t] + model.VC[i, t] * model.P[i, t] + model.SU[i] * model.SU_var[i, t] + model.SD[i] * model.SD_var[i, t] 
               for i in model.I for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

def output_bounds_rule(model, i, t):
    return model.L[i] * model.x[i, t] <= model.P[i, t] <= model.U[i] * model.x[i, t]
model.output_bounds = pyo.Constraint(model.I, model.T, rule=output_bounds_rule)

def ramp_rate_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] <= model.Delta_P_plus[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_up = pyo.Constraint(model.I, model.T, rule=ramp_rate_up_rule)

def ramp_rate_down_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] >= -model.Delta_P_minus[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_down = pyo.Constraint(model.I, model.T, rule=ramp_rate_down_rule)

def startup_rule(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.x[i, t] - model.x[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.x[i, t-1] - model.x[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Optimal Solution:")
for i in model.I:
    for t in model.T:
        print(f"Power Plant {i}, Time {t}: Output = {pyo.value(model.P[i, t])}, Status = {'Running' if pyo.value(model.x[i, t]) == 1 else 'Off'}")

ERROR: Rule failed when generating expression for Constraint output_bounds
with index (1, 1): PyomoException: Cannot convert non-constant Pyomo
expression (10*x[1,1]  <=  P[1,1]) to bool. This error is usually caused by
using a Var, unit, or mutable Param in a Boolean context such as an "if"
statement, or when checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'output_bounds' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression
        (10*x[1,1]  <=  P[1,1]) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pa

PyomoException: Cannot convert non-constant Pyomo expression (10*x[1,1]  <=  P[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [551]:
import pyomo.environ as pyo

# Sample data (replace with actual data)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 2000, 2: 3000, 3: 2500, 4: 4000, 5: 3500, 6: 4500} # Sample data
VC = VC = {(1, 1): 20, (1, 2): 22, (1, 3): 23, (1, 4): 24, (1, 5): 25, (1, 6): 26, (1, 7): 27, (1, 8): 28, (1, 9): 29, (1, 10): 30, (1, 11): 31, (1, 12): 32, (1, 13): 33, (1, 14): 34, (1, 15): 35, (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (2, 10): 24, (2, 11): 25, (2, 12): 26, (2, 13): 27, (2, 14): 28, (2, 15): 29, (3, 1): 18, (3, 2): 19, (3, 3): 20, (3, 4): 21, (3, 5): 22, (3, 6): 23, (3, 7): 24, (3, 8): 25, (3, 9): 26, (3, 10): 27, (3, 11): 28, (3, 12): 29, (3, 13): 30, (3, 14): 31, (3, 15): 32, (4, 1): 25, (4, 2): 26, (4, 3): 27, (4, 4): 28, (4, 5): 29, (4, 6): 30, (4, 7): 31, (4, 8): 32, (4, 9): 33, (4, 10): 34, (4, 11): 35, (4, 12): 36, (4, 13): 37, (4, 14): 38, (4, 15): 39, (5, 1): 22, (5, 2): 23, (5, 3): 24, (5, 4): 25, (5, 5): 26, (5, 6): 27, (5, 7): 28, (5, 8): 29, (5, 9): 30, (5, 10): 31, (5, 11): 32, (5, 12): 33, (5, 13): 34, (5, 14): 35, (5, 15): 36, (6, 1): 30, (6, 2): 31, (6, 3): 32, (6, 4): 33, (6, 5): 34, (6, 6): 35, (6, 7): 36, (6, 8): 37, (6, 9): 38, (6, 10): 39, (6, 11): 40, (6, 12): 41, (6, 13): 42, (6, 14): 43, (6, 15): 44} # Sample data

L = {1: 50, 2: 40, 3: 30, 4: 60, 5: 55, 6: 65} # Sample lower bounds
U = {1: 500, 2:  600, 3: 550, 4: 700, 5: 650, 6: 750}  # Sample upper bounds
Delta_P_plus = {1: 100, 2: 120, 3: 110, 4: 130, 5: 125, 6: 140}  # Sample max power increment
Delta_P_minus = {1: 90, 2: 110, 3: 100, 4: 120, 5: 115, 6: 130} # Sample max power decrement
D = {1: 1000, 2: 1200, 3: 1300, 4: 1100, 5: 1500, 6: 1400, 7: 1600, 8: 1300, 9: 1700, 10: 1800,
     11: 1900, 12: 1600, 13: 2000, 14: 1800, 15: 1700}  # Sample demand data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Set of power plants
model.T = pyo.Set(initialize=range(1, 16))  # Set of time periods

# Parameters (already defined as sample data above)
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.L = pyo.Param(model.I, initialize=L)
model.U = pyo.Param(model.I, initialize=U)
model.Delta_P_plus = pyo.Param(model.I, initialize=Delta_P_plus)
model.Delta_P_minus = pyo.Param(model.I, initialize=Delta_P_minus)
model.D = pyo.Param(model.T, initialize=D)

# Decision variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.x = pyo.Var(model.I, model.T, within=pyo.Binary)  # Running status
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status

# Objective function
def objective_rule(model):
    return sum(model.FC[i] * model.x[i, t] + model.VC[i, t] * model.P[i, t] + model.SU[i] * model.SU_var[i, t] + model.SD[i] * model.SD_var[i, t] 
               for i in model.I for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

def lower_bound_rule(model, i, t):
    return model.L[i] * model.x[i, t] <= model.P[i, t]

def upper_bound_rule(model, i, t):
    return model.P[i, t] <= model.U[i] * model.x[i, t]

model.lower_output_bound = pyo.Constraint(model.I, model.T, rule=lower_bound_rule)
model.upper_output_bound = pyo.Constraint(model.I, model.T, rule=upper_bound_rule)
def ramp_rate_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] <= model.Delta_P_plus[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_up = pyo.Constraint(model.I, model.T, rule=ramp_rate_up_rule)

def ramp_rate_down_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] >= -model.Delta_P_minus[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_down = pyo.Constraint(model.I, model.T, rule=ramp_rate_down_rule)

def startup_rule(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.x[i, t] - model.x[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.x[i, t-1] - model.x[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Optimal Solution:")
for i in model.I:
    for t in model.T:
        print(f"Power Plant {i}, Time {t}: Output = {pyo.value(model.P[i, t])}, Status = {'Running' if pyo.value(model.x[i, t]) == 1 else 'Off'}")


Problem: 
- Name: unknown
  Lower bound: 802166.0
  Upper bound: 802166.0
  Number of objectives: 1
  Number of constraints: 531
  Number of variables: 360
  Number of nonzeros: 1290
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 945
      Number of created subproblems: 945
  Error rc: 0
  Time: 0.8288273811340332
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
Power Plant 1, Time 1: Output = 100.0, Status = Running
Power Plant 1, Time 2: Output = 200.0, Status = Running
Power Plant 1, Time 3: Output = 300.0, Status = Running
Power Plant 1, Time 4: Output = 400.0, Status = Running
Power Plant 1, Time 5: Output = 500.0, Status = Running
Power Plant 1, Time 6: Output = 410.0, Status = Running
Power Plant 1, Time 7: Output = 490.0, Status = Running
Power Plant 1, Time 8: Output = 400.0, Status = Running
Power Plant 1, Time 9: Output = 500.0, Statu

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [553]:
print(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Sets and Indices:**

*  *I* = Set of power plants, *I* = {1, 2, 3, 4, 5, 6}
* *T* = Set of time periods, *T* = {1, 2, ..., 15}

**Parameters:**

*  *SU<sub>i</sub>* = Startup cost for power plant *i* (€), *i* ∈ *I*
    *  *SU<sub>1</sub>* = 10324, *SU<sub>2</sub>* = 5678, *SU<sub>3</sub>* = 7802, *SU<sub>4</sub>* = 12899, *SU<sub>5</sub>* = 4596, *SU<sub>6</sub>* = 9076
* *SD<sub>i</sub>* = Shutdown cost for power plant *i* (€), *i* ∈ *I*
    * *SD<sub>1</sub>* = 2673, *SD<sub>2</sub>* = 5893, *SD<sub>3</sub>* = 982, *SD<sub>4</sub>* = 6783, *SD<sub>5</sub>* = 2596, *SD<sub>6</sub>* = 3561
* *FC<sub>i</sub>* = Fixed cost for running power plant *i* (€/period), *i* ∈ *I* 
* *VC<sub>it</sub>* = Variable cost for power plant *i* in time period *t* (€/MWh), *i* ∈ *I*, *t* ∈ *T*
* *L<sub>i</sub>* = Lower bound for output power of power plant *i* (MW), *i* ∈ *I*
* *U<sub>i</sub>* = Upper bound for output power of power plant *i

In [554]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982,  4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 120, 5: 70, 6: 110}  # Sample fixed costs
VC = {(i, t): 0.1*i for i in range(1, 7) for t in range(1, 16)}  # Sample variable costs
L = {1: 10, 2: 8, 3: 12, 4: 15, 5: 9, 6: 11}  # Sample lower bounds
U = {1: 50, 2: 40, 3: 45, 4: 60, 5: 35, 6: 55}  # Sample upper bounds
Delta_P_plus = {1: 5, 2: 4, 3: 6, 4: 8, 5: 3, 6: 7}  # Sample max power increment
Delta_P_minus = {1: 5, 2: 4, 3: 6, 4: 8, 5: 3, 6: 7}  # Sample max power decrement
D = {t: 100 + 10*t for t in range(1, 16)}  # Sample demand data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Set of power plants
model.T = pyo.Set(initialize=range(1, 16))  # Set of time periods

# Parameters (already defined as sample data above)
model.SU = pyo.Param(model.I, i